# Spark SQL
创建SparkSession对象

In [1]:
# Creating a SparkSession in Python
from pyspark.sql import SparkSession
# getOrCreate()如果存在则获取，如果不存在则创建
spark = SparkSession.builder.master("local").appName("Spark SQL").getOrCreate() 

## 从JSON文件中创建DataFrame

In [17]:
df_json = spark.read.json("./data/people.json")
# show() 打印DataFrame中的所有内容
df_json.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



## 从CSV文件中创建

In [50]:
df_csv = spark.read.csv('./data/people.csv', sep = ',', header = True)

In [19]:
df_csv.show()

+-----+---+---------+
| name|age|      job|
+-----+---+---------+
|Jorge| 30|Developer|
|  Bob| 32|Developer|
+-----+---+---------+



## 通过反射机制，将RDD转换为DataFrame
Spark SQL能够将含Row对象的RDD转换成DataFrame，并推断数据类型。<br>
通过将一个键值对（key/value）列表作为kwargs传给Row类来构造Rows。<br>
key定义了表的列名，类型通过看第一列数据来推断。所以这里RDD的第一列数据不能有缺失。

In [54]:
from pyspark.sql.types import Row
def f(x):
    rel = {}
    rel['name'] = x[0]
    rel['age'] = x[1]
    return rel


df_reflect = spark.sparkContext.textFile("./data/people.txt")\
                .map(lambda line : line.split(','))\
                .map(lambda x: Row(**f(x))).toDF()
df_reflect.show()

+---+-------+
|age|   name|
+---+-------+
| 29|Michael|
| 30|   Andy|
| 19| Justin|
+---+-------+



## 使用编程方式创建DataFrame

In [70]:
from pyspark.sql.types import Row
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import StringType
 
# 读取txt生成RDD
rdd = spark.sparkContext.textFile('./data/people.txt')\
           .map(lambda line: line.split(','))

# 定义Schema中的字段
# StructField(fieldName, DataType(), nullable = True)
schema = StructType([
    StructField("name", StringType(), True),
    StructField("age", StringType(), True)
])

 
# Apply the schema to the RDD and Create DataFrame
df = spark.createDataFrame(rdd, schema)
df.show()

+-------+---+
|   name|age|
+-------+---+
|Michael| 29|
|   Andy| 30|
| Justin| 19|
+-------+---+



In [56]:
df.show()

+-------+---+
|   name|age|
+-------+---+
|Michael| 29|
|   Andy| 30|
| Justin| 19|
+-------+---+



In [21]:
df_json.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [22]:
# DataFrame 常见操作
# 打印DataFrame的模式
df_json.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [24]:
# DataFrame列的选择，并且支持表达式
df_json.select(df_json.name, df_json.age + 1).show()

+-------+---------+
|   name|(age + 1)|
+-------+---------+
|Michael|     null|
|   Andy|       31|
| Justin|       20|
+-------+---------+



In [26]:
# DataFrame 条件过滤
df_json.filter(df_json.age > 20 ).show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



In [27]:
# 分组聚合
df_json.groupBy('age').count().show()

+----+-----+
| age|count|
+----+-----+
|  19|    1|
|null|    1|
|  30|    1|
+----+-----+



In [28]:
# 排序
df_json.sort(df_json.age.desc()).show()

+----+-------+
| age|   name|
+----+-------+
|  30|   Andy|
|  19| Justin|
|null|Michael|
+----+-------+



In [30]:
# 多列排序
df_json.sort(df_json.age.desc(), df_json.name.asc()).show()

+----+-------+
| age|   name|
+----+-------+
|  30|   Andy|
|  19| Justin|
|null|Michael|
+----+-------+



In [35]:
# 对列进行重命名
df_json.select(df_json.name.alias("username"), df_json.age).show()

+--------+----+
|username| age|
+--------+----+
| Michael|null|
|    Andy|  30|
|  Justin|  19|
+--------+----+



In [36]:
# 查看DataFrame
df_json.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [37]:
# 查看前两行
df_json.head(2)

[Row(age=None, name='Michael'), Row(age=30, name='Andy')]

In [38]:
# 查看列名
df_json.columns  

['age', 'name']

In [39]:
# 查看统计信息
df_json.describe().show()  #查看内容---描述统计值（最大，最小，频数等）

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



In [40]:
df_json.describe('age').show() #查看其中一列

+-------+------------------+
|summary|               age|
+-------+------------------+
|  count|                 2|
|   mean|              24.5|
| stddev|7.7781745930520225|
|    min|                19|
|    max|                30|
+-------+------------------+



In [41]:
# 筛选特定样本
# 选择name是'Michael'的值
df_json.filter(df_json['name'] == 'Michael').show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
+----+-------+



In [42]:
# 只显示name那一列
df_json.filter(df_json['name'] == 'Michael').select('name').show()

+-------+
|   name|
+-------+
|Michael|
+-------+



In [44]:
# 去除重复
df_json.select('name').distinct().show()
# 计算不重复值有多少
df_json.select('name').distinct().count()

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
+-------+



3

In [45]:
# 处理缺失值
# 缺失值用0代替
df_json.fillna(0).show()

+---+-------+
|age|   name|
+---+-------+
|  0|Michael|
| 30|   Andy|
| 19| Justin|
+---+-------+



In [46]:
# 直接删除缺失值
df_json.dropna().show() 

+---+------+
|age|  name|
+---+------+
| 30|  Andy|
| 19|Justin|
+---+------+



In [49]:
# 转换类型
from pyspark.sql.types import DoubleType
df_json = df_json.withColumn('age', df_json['age'].cast(DoubleType()))
df_json.printSchema()
df_json.show()

root
 |-- age: double (nullable = true)
 |-- name: string (nullable = true)

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|30.0|   Andy|
|19.0| Justin|
+----+-------+



In [72]:
df_json.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|30.0|   Andy|
|19.0| Justin|
+----+-------+



In [73]:
# 注册DataFrame为临时表， 可以进行SQL的查询
df_json.createOrReplaceTempView("people")
spark.sql("select * from people").show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|30.0|   Andy|
|19.0| Justin|
+----+-------+



In [74]:
spark.sql("select * from people where name = 'Andy'").show()

+----+----+
| age|name|
+----+----+
|30.0|Andy|
+----+----+



In [90]:
res_df = spark.sql("select * from people where name = 'Andy'")
res_df.rdd.map( lambda attributes : (1 + attributes[0] , attributes[1] + 'a')).collect()

[(31.0, 'Andya')]